# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,saint-joseph,-21.3667,55.6167,25.21,92,98,6.65,RE,1706462127
1,1,punta arenas,-53.1500,-70.9167,16.06,51,75,12.35,CL,1706462075
2,2,margaret river,-33.9500,115.0667,13.24,78,71,4.07,AU,1706462256
3,3,bethel,41.3712,-73.4140,3.44,86,100,5.14,US,1706462257
4,4,port-aux-francais,-49.3500,70.2167,5.57,72,89,10.57,TF,1706462257


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

map_plot_1 = city_data_df.hvplot.points(
    'Lng',
    'Lat',
    geo = True,
    tiles = "OSM",
    frame_width = 1200,
    frame_height = 800,
    color = "City",
    size = "Humidity",
    alpha = .5
    
    
)
# Display the map
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
perfect_df_nas = city_data_df[
                 (city_data_df["Max Temp"] < 27) & (city_data_df["Max Temp"] > 21) &
                 (city_data_df["Wind Speed"] < 4.5) &
                 (city_data_df["Cloudiness"] == 0)
]

# Drop any rows with null values
perfect_df = perfect_df_nas.dropna(how='any')

# Display sample data
perfect_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
16,16,san marcos,33.1434,-117.1661,22.13,43,0,1.79,US,1706462172
27,27,hawaiian paradise park,19.5933,-154.9731,21.13,86,0,2.06,US,1706462260
73,73,east london,-33.0153,27.9116,25.52,88,0,2.57,ZA,1706462266
120,120,hilo,19.7297,-155.0900,25.71,84,0,2.06,US,1706462037
243,243,port augusta,-32.5000,137.7667,24.35,38,0,3.04,AU,1706462288
275,275,tindouf,27.6711,-8.1474,23.25,15,0,2.57,DZ,1706462294
450,450,cabo san lucas,22.8909,-109.9124,26.45,69,0,1.79,MX,1706462161
532,532,hassi messaoud,31.6804,6.0729,21.92,20,0,2.06,DZ,1706462330
556,556,broken hill,-31.9500,141.4333,23.95,31,0,3.29,AU,1706462119


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df =perfect_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
16,san marcos,US,33.1434,-117.1661,43,
27,hawaiian paradise park,US,19.5933,-154.9731,86,
73,east london,ZA,-33.0153,27.9116,88,
120,hilo,US,19.7297,-155.0900,84,
243,port augusta,AU,-32.5000,137.7667,38,
275,tindouf,DZ,27.6711,-8.1474,15,
450,cabo san lucas,MX,22.8909,-109.9124,69,
532,hassi messaoud,DZ,31.6804,6.0729,20,
556,broken hill,AU,-31.9500,141.4333,31,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 50000
params = {
    "apiKey":geoapify_key,
    "format":"json",
    "categories":"accommodation.hotel"
    }

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
        
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
san marcos - nearest hotel: Fairfield Inn & Suites by Marriott San Diego North/San Marcos
hawaiian paradise park - nearest hotel: Lava Tree Tropic Inn
east london - nearest hotel: No hotel found
hilo - nearest hotel: Dolphin Bay Hotel
port augusta - nearest hotel: Hotel Commonwealth
tindouf - nearest hotel: محمد بوسبي
cabo san lucas - nearest hotel: Comfort Rooms
hassi messaoud - nearest hotel: AMC
broken hill - nearest hotel: Ibis Styles


,City,Country,Lat,Lng,Humidity,Hotel Name
16,san marcos,US,33.1434,-117.1661,43,Fairfield Inn & Suites by Marriott San Diego N...
27,hawaiian paradise park,US,19.5933,-154.9731,86,Lava Tree Tropic Inn
73,east london,ZA,-33.0153,27.9116,88,No hotel found
120,hilo,US,19.7297,-155.0900,84,Dolphin Bay Hotel
243,port augusta,AU,-32.5000,137.7667,38,Hotel Commonwealth
275,tindouf,DZ,27.6711,-8.1474,15,محمد بوسبي
450,cabo san lucas,MX,22.8909,-109.9124,69,Comfort Rooms
532,hassi messaoud,DZ,31.6804,6.0729,20,AMC
556,broken hill,AU,-31.9500,141.4333,31,Ibis Styles


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    'Lng',
    'Lat',
    geo = True,
    tiles = "OSM",
    frame_width = 1200,
    frame_height = 600,
    color = "City",
    size = "Humidity",
    alpha = .7,
    hover_cols = ["Country","Hotel Name"],
    legend = "right"
)

# Display the map
map_plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)